In [1]:
import requests
import pandas as pd
import urllib.request
import json
import os
from google.cloud import storage

os.environ['GOOGLE_APPLICATION_CREDENTIALS']="ServiceKey_GoogleCloud.json"

# Housing Violations

In [2]:
url = "https://data.cityofnewyork.us/resource/wvxf-dwi5.csv"
housing_raw = pd.read_csv(url)

In [3]:
housing_raw.dtypes

violationid                int64
buildingid                 int64
registrationid             int64
boroid                     int64
boro                      object
housenumber               object
lowhousenumber            object
highhousenumber           object
streetname                object
streetcode                 int64
zip                        int64
apartment                 object
story                     object
block                      int64
lot                        int64
class                     object
inspectiondate            object
approveddate              object
originalcertifybydate     object
originalcorrectbydate     object
newcertifybydate          object
newcorrectbydate          object
certifieddate             object
ordernumber                int64
novid                    float64
novdescription            object
novissueddate             object
currentstatusid            int64
currentstatus             object
currentstatusdate         object
novtype   

# HOUSING DATABASE

In [4]:
url2 = "https://data.cityofnewyork.us/resource/48dt-mn3z.csv"
housingdb_raw = pd.read_csv(url2)


In [5]:
housingdb_raw.head()

,the_geom,cdta2020,cdtaname20,comp2010ap,comp2010,comp2011,comp2012,comp2013,comp2014,comp2015,...,comp2020,comp2021,comp2022,filed,approved,permitted,withdrawn,inactive,shape_leng,shape_area
0,MULTIPOLYGON (((-73.95439555373027 40.73911477...,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),2093.0,3044.0,1351.0,1199.0,1877.0,1425.0,1628.0,...,1646.0,750.0,1974.0,1319.0,811.0,9832.0,2573.0,2607.0,65659.421505,1.316592e+08
1,MULTIPOLYGON (((-73.96929296370487 40.70709333...,BK02,BK02 Downtown Brooklyn-Fort Greene (CD 2 Appro...,803.0,1081.0,847.0,536.0,557.0,1216.0,1844.0,...,1462.0,1838.0,1139.0,159.0,66.0,7632.0,1090.0,404.0,65236.059863,7.592282e+07
2,MULTIPOLYGON (((-73.91804606980472 40.68721324...,BK03,BK03 Bedford-Stuyvesant (CD 3 Approximation),292.0,343.0,569.0,521.0,531.0,321.0,568.0,...,1029.0,1089.0,949.0,620.0,524.0,4345.0,1430.0,773.0,37550.857737,7.819640e+07
3,MULTIPOLYGON (((-73.89652644258406 40.68240329...,BK04,BK04 Bushwick (CD 4 Equivalent),368.0,471.0,264.0,141.0,280.0,250.0,424.0,...,719.0,496.0,526.0,506.0,287.0,2185.0,973.0,862.0,36981.727110,5.665321e+07
4,MULTIPOLYGON (((-73.88828531378107 40.64672241...,BK05,BK05 East New York-Cypress Hills (CD 5 Approxi...,396.0,412.0,380.0,324.0,222.0,32.0,462.0,...,1224.0,1315.0,498.0,1173.0,438.0,4129.0,424.0,586.0,78801.890320,1.654563e+08


# SCHOOL FOOD

In [15]:
url3 = "https://data.cityofnewyork.us/resource/5ery-qagt.csv"
raw_food = pd.read_csv(url3)

In [16]:
nyc_food = raw_food[["schoolname","lastinspection","permittee","ptet","site_type","level","code","violationdescription","latitude","longitude","councildistrict","nta"]]

In [17]:
nyc_food.dtypes

schoolname               object
lastinspection           object
permittee                object
ptet                      int64
site_type                object
level                    object
code                     object
violationdescription     object
latitude                float64
longitude               float64
councildistrict         float64
nta                      object
dtype: object

In [18]:
nyc_food = nyc_food.dropna(subset=['councildistrict'])

In [19]:
nyc_food.councildistrict = nyc_food.councildistrict.astype(int)

In [20]:
nyc_food.head()

,schoolname,lastinspection,permittee,ptet,site_type,level,code,violationdescription,latitude,longitude,councildistrict,nta
0,"P.S. 127 MCKINLEY PARK, RALPH A. FABRIZIO SCHO...",2023-02-02T00:00:00.000,NEW YORK CITY DEPARTMENT OF EDUCATION,2530,FSE (Public School Cafeteria),NaN,NaN,NaN,40.624125,-74.018826,43,BK30
2,P.S. 135 SHELDON A. BROOKNER,2023-01-23T00:00:00.000,NEW YORK CITY DEPARTMENT OF EDUCATION,2530,FSE (Public School Cafeteria),C,04A,Food Protection Certificate (FPC) not held by ...,40.653633,-73.933114,41,BK91
3,PAN AMERICAN INTERNATIONAL HIGH SCHOOL AT MONR...,2023-03-09T00:00:00.000,NEW YORK CITY DEPARTMENT OF EDUCATION,2530,FSE (Public School Cafeteria),C,04L,Evidence of mice or live mice in establishment...,40.831594,-73.878911,18,BX08
4,"PS811M @ PS 149M, SUCCESS ACADEMY CHARTER-HARL...",2023-01-31T00:00:00.000,NEW YORK CITY DEPARTMENT OF EDUCATION,2530,FSE (Public School Cafeteria),C,04M,Live roaches in facility's food or non-food area.,40.802464,-73.946715,9,MN11
5,"P.S. 244 RICHARD R. GREEN, BROOKLYN SCIENCE AN...",2023-01-23T00:00:00.000,NEW YORK CITY DEPARTMENT OF EDUCATION,2530,FSE (Public School Cafeteria),C,04L,Evidence of mice or live mice in establishment...,40.648413,-73.925977,45,BK96


# Congressional District SHAPEFILE

In [39]:
with urllib.request.urlopen("https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/NYC_City_Council_Districts/FeatureServer/0?f=pjson") as url4:
    cd_data = json.load(url4)

# SYEP

In [35]:
url5 = "https://data.cityofnewyork.us/resource/73rz-5b7x.csv"
syep = pd.read_csv(url5)

In [36]:
syep = syep.drop([51])
syep.council_district = syep.council_district.astype(int)

# Construction Demolition

In [40]:
url6 = "https://data.cityofnewyork.us/resource/cspg-yi7g.csv"
demolition_raw = pd.read_csv(url6)

In [41]:
demolition_raw.dtypes

created              object
bic_number           object
account_name         object
trade_name           object
address              object
city                 object
state                object
postcode             object
phone                object
email                object
application_type     object
disposition_date     object
effective_date       object
expiration_date      object
renewal                bool
export_date          object
latitude            float64
longitude           float64
community_board     float64
council_district    float64
census_tract        float64
bin                 float64
bbl                 float64
nta                  object
boro                 object
dtype: object

In [42]:
demolition = demolition_raw[["created","account_name","trade_name","address","phone","email","application_type","disposition_date","effective_date","expiration_date","renewal","export_date","latitude","longitude","community_board","council_district","nta"]]

In [43]:
demo_nyc = demolition.dropna(subset=["council_district","longitude","latitude"])
demo_nyc

,created,account_name,trade_name,address,phone,email,application_type,disposition_date,effective_date,expiration_date,renewal,export_date,latitude,longitude,community_board,council_district,nta
0,2023-08-10T15:34:09.000,J.F.K. CONSTRUCTION SERVICES CORP.,NaN,48-08 70TH ST.,(718) 458-7451,concretejfk@aol.com,CL2 - Exempt,2023-07-06,2023-02-01,2025-01-31,True,2023-08-10T00:00:00.000,40.738169,-73.894126,2.0,30.0,Elmhurst-Maspeth
2,2023-08-10T15:34:09.000,MAGNUS GS INC,NaN,37-19 55TH STREET,646-621-7990,magnus.gsinc@gmail.com,CL2 - Exempt,2023-05-05,2022-12-01,2024-11-30,True,2023-08-10T00:00:00.000,40.750548,-73.907132,2.0,26.0,Woodside
3,2023-08-10T15:34:09.000,"J AND R CAMPO CONSTRUCTION, INC.",NaN,109 BAY 37TH STREET,(718) 449-3430,joe@jandrcampoconstruction.com,CL2 - Exempt,2022-07-29,2022-07-01,2024-06-30,True,2023-08-10T00:00:00.000,40.596477,-73.991059,11.0,47.0,Bensonhurst East
4,2023-08-10T15:34:09.000,CARLOS ARMANDO QUIZHIP CHIMA,C A Q SERVICE,12-04 BOYNTON AVENUE APT. 3C,(410) 800-5214,carlosquizhpiapostol@gmail.com,CL2 - Exempt,NaN,2023-04-01,2025-03-31,True,2023-08-10T00:00:00.000,40.829127,-73.878331,9.0,18.0,West Farms-Bronx River
6,2023-08-10T15:34:09.000,ADONAI CONSTRUCTION CORP.,NaN,157-02 109 AVENUE,(347) 558-0570,adonaiconstruction12@gmail.com,CL2 - Exempt,2023-04-21,2022-03-01,2024-02-29,True,2023-08-10T00:00:00.000,40.693209,-73.794251,12.0,28.0,South Jamaica
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,2023-08-10T15:34:09.000,TRI STATE DISMANTLING CORP.,NaN,207 DUPONT STREET,(718) 349-2552,andre@gotsd.com,CL2 - Exempt,NaN,2022-04-01,2024-03-31,True,2023-08-10T00:00:00.000,40.736264,-73.951142,1.0,33.0,Greenpoint
994,2023-08-10T15:34:09.000,GLASS ARMOR OF NEW YORK INC.,EMPIRE ENVIRONMENTAL SERVICES,12 BAY 49TH STREET,(718) 714-6898,info@empireenvironmental.com,CL2 - Exempt,2022-12-13,2021-11-01,2023-10-31,True,2023-08-10T00:00:00.000,40.589238,-73.984266,13.0,47.0,Gravesend
996,2023-08-10T15:34:09.000,CRESCENT STREET CONSTRUCTION CORPORATION,NaN,104-22 ASTORIA BLVD,(718) 706-8500,jimmyv@crescentgc.com,CL2 - Exempt,2023-08-09,2022-08-01,2024-07-31,True,2023-08-10T00:00:00.000,40.761241,-73.864973,3.0,21.0,East Elmhurst
997,2023-08-10T15:34:09.000,BARCELONA CONTRACTING CORP.,NaN,107-28 156TH STREET,(718) 558-8686,barcelonacontracting1@gmail.com,CL2 - Exempt,2023-03-29,2021-12-01,2023-11-30,True,2023-08-10T00:00:00.000,40.695445,-73.797086,12.0,28.0,South Jamaica


# DOB Permits

In [44]:
url7= "https://data.cityofnewyork.us/resource/ipu4-2q9a.csv"
permit_raw = pd.read_csv(url7)

In [45]:
permit_raw.dtypes

borough                              object
bin__                                 int64
house__                              object
street_name                          object
job__                                 int64
job_doc___                            int64
job_type                             object
self_cert                            object
block                                 int64
lot                                   int64
community_board                       int64
zip_code                              int64
bldg_type                           float64
residential                          object
special_district_1                   object
special_district_2                  float64
work_type                            object
permit_status                        object
filing_status                        object
permit_type                          object
permit_sequence__                     int64
permit_subtype                       object
oil_gas                         

In [46]:
permit_raw["expiration_date"]

0      05/10/2023
1      10/01/2022
2      10/01/2022
3      10/01/2022
4      11/15/2022
          ...    
995    09/16/2022
996    05/13/2023
997    10/01/2022
998    03/24/2023
999    12/24/2022
Name: expiration_date, Length: 1000, dtype: object

# Sending data to BigQuery

In [47]:
storage_client = storage.Client()
bucket = storage_client.get_bucket('valley-bucket')

In [48]:
def upload_to_bucket(blob_name, file_path):
    try:
        blob = bucket.blob(blob_name)
        blob.upload_from_string(file_path, 'text/csv')
        return True
    except Exception as e:
        print(e)
        return False

In [53]:
def create_json(json_object, filename):
    blob = bucket.blob(filename)
    blob.upload_from_string(
        data=json.dumps(json_object),
        content_type='application/json'
        )
    result = filename + ' upload complete'
    return {'response' : result}

In [49]:
upload_to_bucket("SYEP",syep.to_csv())

True

In [50]:
upload_to_bucket("School Foods",nyc_food.to_csv())

True

In [55]:
print(create_json(cd_data, "Council Districts"))

{'response': 'Council Districts upload complete'}
